In [ ]:
import numpy as np 
import xgboost as xgb
from xgboost.plotting import plot_importance

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, average_precision_score, cohen_kappa_score
#from sacred import Experiment
#from sacred.observers import MongoObserver
#from sacred.observers import FileStorageObserver
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler 
from sklearn.utils import shuffle

In [ ]:
#Define training vars
basic_vars = ['met_met', 'met_phi',
            'lead_lep_pt', 'lead_lep_eta', 'lead_lep_phi','lead_lep_e',
             'jet_pt[0]', 'jet_eta[0]', 'jet_phi[0]', 'jet_e[0]','isbtagged_MV2c10_85[0]',
             'jet_pt[1]', 'jet_eta[1]', 'jet_phi[1]', 'jet_e[1]','isbtagged_MV2c10_85[1]']

In [ ]:
mva = 'DL1_85'
#Define full vars
train_vars = ['met_met', 'met_phi',
            'lead_lep_pt', 'lead_lep_eta', 'lead_lep_phi','lead_lep_e',
             'jet_pt[0]', 'jet_eta[0]', 'jet_phi[0]', 'jet_e[0]', 'jet_isbtagged_%s[0]'%(mva),
             'jet_pt[1]', 'jet_eta[1]', 'jet_phi[1]', 'jet_e[1]', 'jet_isbtagged_%s[1]'%(mva),
             'HT_all', 'Centrality_all', 'dEtajl_MaxdEta', 'dRbl_MindPhi_%s'%(mva), 'dRbj_MaxdEta_%s'%(mva), 'dRjl_MindR_%s'%(mva),
             "Aplanarity_jets", 'H0_all', 'nFJets', 'nHFJets', 'nJets', 'nBJets_%s'%(mva), 'HardCentralVeto_%s'%(mva), 'H2_jets', 'VLQMass_Lead_Jet']

In [ ]:
train_vars = ['met_met',
            'lead_lep_pt','lead_lep_eta','lead_lep_e',
             'jet_pt[0]', 'jet_eta[0]',  'jet_e[0]', 'jet_isbtagged_%s[0]'%(mva),
             'jet_pt[1]', 'jet_eta[1]',  'jet_e[1]', 'jet_isbtagged_%s[1]'%(mva),
             'HT_all', 'Centrality_all', 'dEtajl_MaxdEta', 'dRbl_MindPhi_%s'%(mva), 'dRbj_MaxdEta_%s'%(mva), 'dRjl_MindR_%s'%(mva),
             "Aplanarity_jets", 'H0_all', 'nFJets', 'nJets', 'nBJets_%s'%(mva), 'HardCentralVeto_%s'%(mva), 'H2_jets', 'VLQMass_Lead_Jet']

In [ ]:
train_vars2 = ['met_met',
            'lead_lep_pt','lead_lep_eta','lead_lep_e',
             'jet_pt[0]', 'jet_eta[0]',  'jet_e[0]', 'jet_isbtagged_%s[0]'%(mva),
             'jet_pt[1]', 'jet_eta[1]',  'jet_e[1]', 'jet_isbtagged_%s[1]'%(mva),
             'HT_all', 'Centrality_all', 'dEtajl_MaxdEta', 'dRbl_MindPhi_%s'%(mva), 'dRbj_MaxdEta_%s'%(mva),
             "Aplanarity_jets", 'H0_all', 'nFJets', 'nJets', 'nBJets_%s'%(mva), 'HardCentralVeto_%s'%(mva), 'H2_jets', 'VLQMass_Lead_Jet']

In [ ]:
mva = 'DL1_77'
full_vars = ['met_met', 'met_phi',
                'lead_lep_pt', 'lead_lep_eta', 'lead_lep_phi','lead_lep_e',
                 'jet_pt[0]', 'jet_eta[0]', 'jet_phi[0]', 'jet_e[0]','jet_isbtagged_%s[0]'%(mva),
                 'jet_pt[1]', 'jet_eta[1]', 'jet_phi[1]', 'jet_e[1]','jet_isbtagged_%s[1]'%(mva),
                 'dRbl_MindR_%s'%(mva), "dPhiMET_LeadBJet_%s"%(mva), "dPhiW_LeadB_%s"%(mva), "dPhiMETlep", "dRjl_MindR", "pT_jet3", "dRW_slj",
                 "dPhiMET_SubLeadJet", 'H0_all', 'HT_all', 'Centrality_all', 'H2_jets', 'weight_normalise',
                 'nFJets', 'nFJets40', 'nFJets_fjvt','nFJets_fjvt40', 'nJets', 'nBJets_%s'%(mva), 'HardCentralVeto_%s'%(mva), 'VLQMass_Lead_Jet'] 


train_vars = ['met_met', 
                'lead_lep_pt', 'lead_lep_eta',# 'lead_lep_e',
                 'jet_pt[0]', 'jet_eta[0]','jet_isbtagged_%s[0]'%(mva),
                 'jet_pt[1]', 'jet_eta[1]','jet_isbtagged_%s[1]'%(mva),
                 'dRbl_MindR_%s'%(mva), "dPhiW_LeadB_%s"%(mva), "dPhiMETlep", "dRjl_MindR", "pT_jet3", "dRW_slj",
                 "dPhiMET_SubLeadJet", 'H0_all', 'HT_all', 'Centrality_all', 'H2_jets', 'weight_normalise', 'LeadFJetPt',
                 'nFJets_fjvt', 'nJets', 'nBJets_%s'%(mva), 'HardCentralVeto_%s'%(mva), 
                 'dPhiLep_LeadB_%s'%(mva), 'dRLep_LeadB_%s'%(mva), 'MindRl_centjet', 'dRW_LeadB_%s'%(mva)]# 'VLQMass_Lead_Jet'] 

In [ ]:
#Define training file names
data_dir = "data/"
sgtop_file = data_dir + "sgtop_train_%s.npy" %(mva)  
ttbar_file = data_dir + "ttbar_train_%s.npy" %(mva)   
wjets_file = data_dir + "wjets_train_%s.npy" %(mva) 
zjets_file = data_dir + "zjets_train_%s.npy" %(mva) 
sig_file   = data_dir + "sig_train_%s.npy" %(mva) 

In [ ]:
#Extract training arrays
sgtop_arr = np.load(sgtop_file, encoding="bytes")
ttbar_arr = np.load(ttbar_file, encoding="bytes")
wjets_arr = np.load(wjets_file, encoding="bytes")
zjets_arr = np.load(zjets_file, encoding="bytes")

sig_arr   = np.load(sig_file, encoding="bytes")

In [ ]:
(len(wjets_arr) + len(sgtop_arr) + len(ttbar_arr))/len(sig_arr)

In [ ]:
#Transform into dataframes
sgtop_df = pd.DataFrame(sgtop_arr)
ttbar_df = pd.DataFrame(ttbar_arr)
wjets_df = pd.DataFrame(wjets_arr)
zjets_df = pd.DataFrame(zjets_arr)
sig_df = pd.DataFrame(sig_arr)

In [ ]:
#full['weight_normalise'] = full['weight_normalise']

In [ ]:
sgtop_df.columns

In [ ]:
#Label as sig or back
sgtop_df['label'] = 0
ttbar_df['label'] = 0
wjets_df['label'] = 0
zjets_df['label'] = 0
sig_df['label'] = 1

In [ ]:
#Label as sig or back
sgtop_df['weight'] = sgtop_df['weight_normalise']*len(sgtop_df['weight_normalise'])
ttbar_df['weight'] = ttbar_df['weight_normalise']*len(ttbar_df['weight_normalise'])
wjets_df['weight'] = wjets_df['weight_normalise']*len(wjets_df['weight_normalise'])
zjets_df['weight'] = zjets_df['weight_normalise']*len(zjets_df['weight_normalise'])
sig_df['weight'] =   sig_df['weight_normalise']*len(sig_df['weight_normalise'])



In [ ]:
#Concat arrays, define train and validation sets, create xgb datasets
full = pd.concat([sig_df, sgtop_df, ttbar_df, wjets_df, zjets_df])


In [ ]:
#wjets_df['weight']

In [ ]:
full.columns = [x.replace('[', '_').replace(']', '') for x in full.columns.values]
clean_train_vars = [x.replace('[', '_').replace(']', '') for x in train_vars]

In [ ]:
clean_train_vars = clean_train_vars + ['weight']

In [ ]:
#Select only sig region

#Old SR
#full = full.query('dRjl_MindR > 2.0 & nFJets40 >= 1 & jet_pt_0 > 350000 & jet_isbtagged_%s_0 == 1 & HardCentralVeto_%s == 0 & met_met > 120000 & abs(dPhibl_MindPhi_%s)>2.5'%(mva,mva,mva))

#Upd SR
#full = full.query('dRbl_MindR_%s > 2.0 & nFJets_fjvt40 >= 1 & jet_pt_0 > 350000 & jet_isbtagged_%s_0 == 1 & HardCentralVeto_%s == 0 & met_met > 120000'%(mva,mva,mva))

#Loose SR
#full = full.query('dRbl_MindR_%s > 2.0 & nFJets_fjvt40 >= 1 & met_met > 120000'%(mva))


In [ ]:

#full = full.drop(full['neutrino_eta'].idxmax())

train, valid = train_test_split(full, test_size=0.2)

ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_sample(train[clean_train_vars], train['label'])

X_res, y_res = shuffle(X_res, y_res)





In [ ]:
#test = valid

In [ ]:
res_train = pd.DataFrame(X_res)
res_train.columns = clean_train_vars
res_train['label'] = y_res

In [ ]:
clean_train_vars.remove('weight')#_normalise')
clean_train_vars.remove('weight_normalise')

In [ ]:
dtrain = xgb.DMatrix(res_train[clean_train_vars], label=res_train['label'], weight=res_train['weight'])
dvalid = xgb.DMatrix(valid[clean_train_vars], label=valid['label'], weight=res_train['weight'])

In [ ]:
#clean_train_vars

In [ ]:
#Naive training parameters
naive_params = {'max_depth':10, 
                'eta':0.05, 
                'silent':1, 
                'objective':'binary:logistic', 
                'eval_metric' : 'auc', 
                'tree_method' : 'gpu_hist',#'gpu_hist',
               }
                #'scale_pos_weight': 190 }

In [ ]:
naive_params = {'eta': 0.03,
   'eval_metric': 'auc',
   #'gamma': 8.0,
   #'max_delta_step': 9,
   'max_depth': 6,
   #'max_leaves': 230,
   #'min_child_weight': 12.0,
   'objective': 'binary:logistic',
   'random_state': 42,
   'silent': 1,
   'tree_method': 'hist', 
    'nthread': 10}

In [ ]:
#sig_df['lead_lep_e']/sig_df['lead_lep_pt']

In [ ]:
#%%time
#Train with naive params

bst = xgb.train(naive_params,
                dtrain,
                num_boost_round=5000,
                evals=[(dvalid,"val")],
                early_stopping_rounds=10)

In [ ]:
bst.save_model('011.%s.model'%(mva))
#bst.dump_model('dump.%s.raw.txt'%(mva))# dump model with feature mapz

In [ ]:
ax = plot_importance(bst)
plt.show()

In [ ]:
sgtop_file_t = data_dir + "sgtop_test_%s.npy" %(mva)  
ttbar_file_t = data_dir + "ttbar_test_%s.npy" %(mva)   
zjets_file_t = data_dir + "zjets_test_%s.npy" %(mva) 
wjets_file_t = data_dir + "wjets_test_%s.npy" %(mva) 
sig_file_t   = data_dir + "sig_test_%s.npy" %(mva) 

In [ ]:
#Extract training arrays
sgtop_arr_t = np.load(sgtop_file, encoding="bytes")
ttbar_arr_t = np.load(ttbar_file, encoding="bytes")
wjets_arr_t = np.load(wjets_file, encoding="bytes")
zjets_arr_t = np.load(zjets_file, encoding="bytes")

sig_arr_t   = np.load(sig_file, encoding="bytes")

In [ ]:
#Transform into dataframes
sgtop_dft = pd.DataFrame(sgtop_arr)
ttbar_dft = pd.DataFrame(ttbar_arr)
wjets_dft = pd.DataFrame(wjets_arr)
zjets_dft = pd.DataFrame(zjets_arr)
sig_dft = pd.DataFrame(sig_arr)

In [ ]:
#Label as sig or back
sgtop_dft['label'] = 0
ttbar_dft['label'] = 0
wjets_dft['label'] = 0
zjets_dft['label'] = 0

sig_dft['label'] = 1

In [ ]:
test = pd.concat([sig_dft, sgtop_dft, ttbar_dft, wjets_dft, zjets_dft])

In [ ]:
test.columns = [x.replace('[', '_').replace(']', '') for x in test.columns.values]


In [ ]:
#Old SR
#test = test.query('dRjl_MindR > 2.0 & nFJets40 >= 1 & jet_pt_0 > 350000 & jet_isbtagged_%s_0 == 1 & HardCentralVeto_%s == 0 & met_met > 120000 & abs(dPhibl_MindPhi_%s)>2.5'%(mva,mva,mva))

#Upd SR
#test = test.query('dRbl_MindR_%s > 2.0 & nFJets_fjvt40 >= 1 & jet_pt_0 > 350000 & jet_isbtagged_%s_0 == 1 & HardCentralVeto_%s == 0 & met_met > 120000'%(mva,mva,mva))

#Loose SR
#test = test.query('dRbl_MindR_%s > 2.0 & nFJets_fjvt40 >= 1  & met_met > 120000'%(mva))

In [ ]:
dtest = xgb.DMatrix(test[clean_train_vars], label=test['label'])

In [ ]:
pred_vals = bst.predict(dtest)


In [ ]:
fpr, tpr, _ = roc_curve(dtest.get_label(), pred_vals)
roc_auc = roc_auc_score(dtest.get_label(), pred_vals)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
pred = np.where(pred_vals > np.mean(0.95), 1, 0)

In [ ]:
confusion_matrix(pred, dtest.get_label())

In [ ]:
average_precision_score(pred, dtest.get_label())


In [ ]:
cohen_kappa_score(pred, dtest.get_label())

In [ ]:
roc_auc

In [ ]:
#s/np.sqrt(b)*5/np.sqrt(5)

In [ ]:
np.mean(pred_vals)

In [ ]:
#bst.feature_names

In [ ]:
test['preds']  = pred_vals

In [ ]:
ax = test['preds'][test['label']==1].plot.hist(bins=20, alpha=0.5)
ax = test['preds'][test['label']==0].plot.hist(bins=20, alpha=0.5)

In [ ]:
SR = test.query('dRbl_MindR_%s > 2.0 & nFJets >= 1 & jet_pt_0 > 350000 & jet_isbtagged_%s_0 == 1 & HardCentralVeto_%s == 0 & met_met > 120000'%(mva,mva,mva))

In [ ]:
tt_CR = test.query('nFJets >= 1 & jet_pt_0 > 200000 & jet_isbtagged_%s_0 == 1 & HardCentralVeto_%s == 1 & met_met > 120000'%(mva,mva))

In [ ]:
wj_CR = test.query('dRbl_MindR_%s > 2.0 & nBJets_%s == 1 & jet_pt_0 > 250000 & jet_isbtagged_%s_0 == 1 & HardCentralVeto_%s == 0 & met_met > 120000'%(mva,mva,mva,mva))

In [ ]:
ax = SR['preds'][SR['label']==1].plot.hist(bins=20, alpha=0.5)
ax = SR['preds'][SR['label']==0].plot.hist(bins=20, alpha=0.5)

In [ ]:
ax = SR['preds'][SR['label']==1].plot.hist(bins=20, alpha=0.5)
ax = SR['preds'][SR['label']==0].plot.hist(bins=20, alpha=0.5)

In [ ]:
ax = wj_CR['preds'][wj_CR['label']==1].plot.hist(bins=20, alpha=0.5)
ax = wj_CR['preds'][wj_CR['label']==0].plot.hist(bins=20, alpha=0.5)

In [ ]:
ax = tt_CR['preds'][tt_CR['label']==1].plot.hist(bins=20, alpha=0.5)
ax = tt_CR['preds'][tt_CR['label']==0].plot.hist(bins=20, alpha=0.5)

In [ ]:
test.columns